# Baceterial DEG analysis

## I. Setting workspace

### 0. final directory structure and name of variable

- $(name of variable) # description  
\$(threads) # number of threads available

>```
>User@Machine:~$ ls
>```
>>exp_run/
>```
>User@Machine:~/exp_run$ ls
>```
>>```
>>0_raw/
>>0_tools/
>>0_reference_genome/
>>```

### 1. preparation of the workspace directory

- let`s say that the home directory (~) is /home/username
>```
>User@Machine:/$ cd /home/username
>User@Machine:~$ mkdir exp_run && cd exp_run
>User@Machine:~/exp_run$ pwd
>```
>>```
>>/home/username/exp_run
>>```

### 2. install dependencies

- click name of the tools to access webpage to get more help  

    [NCBI SRA toolkit](https://github.com/ncbi/sra-tools): SRA toolkit helping dealing with SRR files  
    [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/): Quality control and adapter sequence report  
    [cutadapt](https://cutadapt.readthedocs.io/en/stable/): purge adapter sequences  
    [Trim Galore](https://www.bioinformatics.babraham.ac.uk/projects/trim_galore/): trim adapter sequences  
    [Bowtie2](https://github.com/BenLangmead/bowtie2): align reads  
    [samtools](http://www.htslib.org): dealing sam and bam format files  
    [htseq](https://htseq.readthedocs.io/en/master/): count reads aligned on the reference genome  

- install dependencies mentioned above in this directory  
- fastQC, cutadapt, and trim galore can be replaced by fastp instead.  
>```
>User@Machine:~/exp_run$ mkdir tools && cd tools
>User@Machine:~/exp_run/tools$
>```

## II. Counting reads

### 0. prepare SRA file and reference genome

- download SRA from NCBI SRA, or prepare your own fastq file  
>```
>User@Machine:~/exp_run$ mkdir 0_raw && cd 0_raw
>User@Machine:~/exp_run/0_raw$ wget "$(download link for SRA file)"
>User@Machine:~/exp_run/0_raw$ fasterq-dump -I --split-3 $(SRA file)
>User@Machine:~/exp_run/0_raw$ cd ../
>```

- prepare reference genome sequence and annotation
- store the genome files and make sure that you prepared the fasta and gff3 of the reference genome  
>```
>User@Machine:~/exp_run$ mkdir 0_reference_genome && cd 0_reference_genome
>```

- Index the genome to use in bowtie2
>```
>User@Machine:~/exp_run/0_reference_genome$ bowtie2-build --threads $(threads) $(genome fasta) $(output database name)
>User@Machine:~/exp_run/0_reference_genome$ cd ../
>```

- QC for fastq
>```
>User@Machine:~/exp_run$ cd 0_raw/
>User@Machine:~/exp_run/0_raw$ fastqc $(SRA file) -o $(fastqc_output_directory) -t $(threads)
>```

- Quality based trimming and adapter trimming by Trim Galore
>```
>User@Machine:~/exp_run/0_raw$ trim_galore -q $(quality for Trim Galore) --length $(length for Trim Galore) --FastQC --phred33 --paired $(_1.fastq.gz) $(_2.fastq.gz) -o $(trimmed_output_directory)
>User@Machine:~/exp_run/0_raw$ cd ../
>```

### 1. mapping reads

- mapping trimmed reads
>```
>User@Machine:~/username/exp_run$ mkdir 1_align
>User@Machine:~/username/exp_run$ bowtie2 -x 0_reference_genome/$(reference_genome_database) -1 0_raw/$(trimmed_output_directory)/$(_1.fastq.gz) -2 0_raw/$(trimmed_output_directory)/$(_2.fastq.gz) -S 1_align/$(output_samfile) -p $(threads)
>```

### 2. sam to bam, sorting, and flagstat

- convert sam file to bam file
>```
>User@Machine:~/username/exp_run$ samtools view -@ $(threads) -bS $(output_samfile) > $(output_bamfile)
>```

- sort the bam file
>```
>User@Machine:~/username/exp_run$ samtools sort -n $(output_bamfile) -o $(sorted_bamfile)
>```

- make flagstat file
>```
>User@Machine:~/username/exp_run$ samtools flagstat $(sorted_bamfile) > $(sorted_bamfile_flagstatfile)
>```


### 3. count reads

- GTF files can be used as $(reference_gff_file) alternatively.
>```
>User@Machine:~/username/exp_run$ htseq-count -m union -s no -t gene -i gene_id $(output_samfile) $(reference_gff_file) > $(read_countfile)
>```

## III. Normalisation and visualisation using R

### 0. preparing environment

- call libraries in R terminal
>```
>library(DESeq2)
>library(ggplot2)
>library(gplots)
>library(pheatmap)
>library(RColorBrewer)
>```

### 1. 